In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

from userkits.features import *
from userkits.data_loading import load_train_data, load_eval_data

In [ ]:
X, y = load_train_data(data_dir='../train')
X_eval = load_eval_data(data_dir='../eval')

In [ ]:
def extract_features(images):
    features_list = []
    for img in tqdm(images, desc="Extracting features"):
        feats = []
        # add feature functions here
        feats.extend(color_histogram(img))
        feats.extend(lbp_texture_features(img))
        feats.extend(find_mean(img))
        feats.extend(find_stddev(img))

        features_list.append(feats)
    return np.array(features_list)

In [ ]:
X_features = extract_features(X)
X_features.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y, test_size=0.2) # you can change test_size
clf = RandomForestClassifier(verbose=1) # you can tune hyperparameters here
clf.fit(X_train, y_train)
print("Train Accuracy:", clf.score(X_train, y_train))
print("Test Accuracy:", clf.score(X_test, y_test))

## Evaluate

In [ ]:
X_eval_features = extract_features(X_eval)
eval_predictions = clf.predict(X_eval_features)
print(eval_predictions[:5])